In [3]:
import numpy as np
import zarr

from src.common.files import get_raw_paths, get_processed_path
from src.visualization.render_mp4 import unpickle_data, pickle_data
from src.visualization.render_mp4 import create_mp4_jupyter, mp4_from_pickle_jupyter

from furniture_bench.robot.robot_state import filter_and_concat_robot_state


from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt

from tqdm import tqdm

import os
import shutil

In [2]:
raw_paths = get_raw_paths(
    controller="diffik",
    domain="sim",
    demo_outcome="success",
    demo_source="teleop",
    randomness="low",
    task="one_leg",
)

len(raw_paths)

Found the following paths:
    /data/scratch-oc40/pulkitag/ankile/furniture-data/raw/diffik/sim/one_leg/teleop/low/success/**/*.pkl*


40

In [4]:
for i, p in enumerate(raw_paths):
    mp4_from_pickle_jupyter(p, f"demo_{i}.mp4", fps=10, speed_annotation=True)

File saved as demo_0.mp4


File saved as demo_1.mp4


File saved as demo_2.mp4


File saved as demo_3.mp4


File saved as demo_4.mp4


...

File saved as demo_14.mp4


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f3e41032970>>
Traceback (most recent call last):
  File "/data/scratch/ankile/miniconda3/envs/rr/lib/python3.8/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


File saved as demo_15.mp4


File saved as demo_16.mp4


File saved as demo_17.mp4


## Look at videos from the processed data

In [ ]:
import zarr
import numpy as np
from src.common.files import get_processed_path
from src.visualization.render_mp4 import create_mp4_jupyter, annotate_frames_with_speed

In [ ]:
zarr_path = get_processed_path(
    controller="diffik",
    domain="real",
    demo_outcome="success",
    demo_source="teleop",
    randomness="low",
    task="place_shade",
)

print(zarr_path)

z = zarr.open(zarr_path)

list(z.keys())

In [ ]:
ep_ends = z["episode_ends"][:]
print(ep_ends)

images = z["color_image2"][:]

# Split the color images into episodes
ep_images = np.split(images, ep_ends[:-1])

len(ep_images)

In [ ]:
# Calculate the delta position action for each episode
action_pos = z["action/pos"][:, :3]
action_pos = np.split(action_pos, ep_ends[:-1])

robot_pos = z["robot_state"][:, :3]
robot_pos = np.split(robot_pos, ep_ends[:-1])

action_delta_pos = [action - robot for action, robot in zip(action_pos, robot_pos)]

len(action_delta_pos), action_delta_pos[0].shape

In [ ]:
import cv2
import numpy as np


def add_black_space_and_text(
    frames, delta_action, text_color=(255, 255, 255), font_scale=0.5, thickness=1
):
    # Get the dimensions of the frames
    T, H, W, C = frames.shape

    # Define the height of the black space
    black_space_height = 50

    # Create a black space array
    black_space = np.zeros((T, black_space_height, W, C), dtype=np.uint8)

    # Concatenate the frames with the black space
    frames_with_space = np.concatenate((frames, black_space), axis=1)

    # Add text to the black space for each frame
    for i in range(T):
        text = f"{delta_action[i].round(3)}"
        cv2.putText(
            frames_with_space[i],
            text,
            (10, H + black_space_height - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            font_scale,
            text_color,
            thickness,
        )

    return frames_with_space


# Make a video of the episodes
for i, (ep, delta_action) in enumerate(zip(ep_images, action_delta_pos)):
    ep = annotate_frames_with_speed(frames=ep, fps=5)
    ep_with_text = add_black_space_and_text(ep, delta_action)
    create_mp4_jupyter(ep_with_text, f"ep_{i}.mp4", fps=5)

## Look at new 50 one_leg_insert demos

### Raw demos

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.spatial.transform import Rotation as R

from src.visualization.render_mp4 import mp4_from_pickle_jupyter, unpickle_data, pickle_data
from src.common.files import get_raw_paths, get_processed_path

from furniture_bench.robot.robot_state import filter_and_concat_robot_state


import random

In [ ]:
raw_paths = get_raw_paths(
    controller="diffik",
    domain="real",
    demo_outcome="success",
    demo_source="teleop",
    randomness="low",
    task="one_leg_insert",
)

random.shuffle(raw_paths)
raw_paths = sorted(raw_paths, reverse=True)

len(raw_paths)

In [ ]:
for i, raw_path in enumerate(raw_paths[:10]):
    print(f"Showing video {i}, {raw_path}")
    mp4_from_pickle_jupyter(raw_path, filename=f"raw_{i}.mp4", fps=10, speed_annotation=True)

Are the actions position actions in the raw demos?

In [ ]:
data = unpickle_data(raw_paths[0])

data.keys()

In [ ]:
# Extract the robot state and actions
robot_state = np.array(
    [filter_and_concat_robot_state(o["robot_state"]) for o in data["observations"]],
)[:, :3]

actions = np.array(data["actions"])[:, :3]

robot_state.shape, actions.shape

In [ ]:
# Print a few robot states and actions
print(robot_state[:5])
print(actions[:5])

Convert the actions to be delta actions in the raw demos so it's compatible with the data processing pipeline.

In [ ]:
# Change only actions to be delta actions
for raw_path in tqdm(raw_paths):
    data = unpickle_data(raw_path)
    actions = np.array(data["actions"])
    robot_state = np.array(
        [filter_and_concat_robot_state(o["robot_state"]) for o in data["observations"]],
        dtype=np.float32,
    )

    robot_pos, robot_quat, gripper_width = (
        robot_state[:, :3],
        robot_state[:, 3:7],
        robot_state[:, -1:],
    )
    action_pos, action_quat, action_gripper = (
        actions[:, :3],
        actions[:, 3:7],
        actions[:, -1:],
    )

    # Calclate what the delta action is
    action_delta_pos = action_pos - robot_pos

    # Calculate the delta quaternion
    robot_r = R.from_quat(robot_quat)
    action_r = R.from_quat(action_quat)
    action_delta_r = robot_r.inv() * action_r
    action_delta_quat = action_delta_r.as_quat()

    # Concatenate the delta action
    action_delta = np.concatenate(
        [action_delta_pos, action_delta_quat, action_gripper], axis=-1
    )

    data["actions"] = action_delta
    data["furniture"] = "one_leg"
    pickle_data(data, raw_path)

### Processed demos

In [ ]:
processed_path = get_processed_path(
    controller="diffik",
    domain="real",
    demo_outcome="success",
    demo_source="teleop",
    randomness="low",
    task="one_leg_insert",
)

print(processed_path)

z = zarr.open(processed_path)

list(z.keys())

In [ ]:
# Get average episode length
dict(z.attrs)

In [ ]:
# Look at the distribution of the actions
d_action = z["action/delta"][:, :3]

plt.hist(np.linalg.norm(d_action, axis=-1))

Compare the above histogram with the corresponding histogram for one_leg demos in the simulator

In [ ]:
sim_oneleg = get_processed_path(
    controller="diffik",
    domain="sim",
    demo_outcome="success",
    demo_source="teleop",
    randomness="low",
    task="one_leg",
)

print(sim_oneleg)

z_sim_oneleg = zarr.open(sim_oneleg)

list(z_sim_oneleg.keys())

In [ ]:
# Get the first 300 timesteps of each episode
sim_oneleg_ep_ends = z_sim_oneleg["episode_ends"][:]

d_actions = z_sim_oneleg["action/delta"][:, :3]
d_actions = np.split(d_actions, sim_oneleg_ep_ends[:-1])

d_actions = [d_action[:300] for d_action in d_actions]
d_actions = np.concatenate(d_actions)

d_actions.shape

In [ ]:
# Look at the distribution of the actions
d_actions_norm = np.linalg.norm(d_actions, axis=-1)
# plt.hist(d_actions.flatten())
plt.hist(d_actions_norm)